In [1]:
import csv
import datetime
import dateutil.parser
%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4
import numpy
import os

In [44]:
path = os.getcwd()

latitude0 = -9.75
longitude0 = 109.5
lat_resolution = 0.75
lon_resolution = 0.75

onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

csv_file =  open('precip.csv', 'w')
writer = csv.writer(csv_file)
row = ["datetime","latitude","longitude","Precipitation"]
writer.writerow(row)


for f in onlyfiles :
    if f.startswith("2015_") and f.endswith(".nc"):
        print (f)
        tmp = f.replace(".", "_")
        nc_file = netCDF4.Dataset(path + "/"+f,'r')
        arr = tmp.split("_")
        lat_index=int(arr[1])
        long_index=int(arr[2])
        
        
        lat = (0.5-lat_index )*lat_resolution + latitude0
        lng = (long_index - 0.5 )*lon_resolution + longitude0
        
        nc_time = nc_file.variables["time"][:]
        nc_time_units = getattr(nc_file.variables["time"],"units")
        dt_utc = netCDF4.num2date(nc_time,nc_time_units)
        
        for i, t in enumerate(nc_time):
            row = [dt_utc[i],lat,lng,nc_file.variables["Precip"][i]]
            #nc_file.variables["Fsd"][i]]
            #,nc_file.variables["Ta"][i],
            #nc_file.variables["RH"][i],
            #nc_file.variables["Ws"][i],
            #nc_file.variables["Wd"][i],
            #nc_file.variables["Precip"][i]]
            writer.writerow(row)
csv_file.close()

2015_10_48.nc
2015_11_18.nc
2015_12_17.nc
2015_12_19.nc
2015_12_20.nc
2015_12_25.nc
2015_12_31.nc
2015_12_32.nc
2015_12_38.nc
2015_12_42.nc
2015_13_23.nc
2015_13_33.nc
2015_13_34.nc
2015_14_10.nc
2015_14_15.nc
2015_14_30.nc
2015_14_52.nc
2015_15_12.nc
2015_15_24.nc
2015_15_27.nc
2015_15_40.nc
2015_15_49.nc
2015_15_52.nc
2015_16_11.nc
2015_16_13.nc
2015_16_28.nc
2015_16_29.nc
2015_16_50.nc
2015_16_9.nc
2015_17_14.nc
2015_17_51.nc
2015_17_53.nc
2015_18_21.nc
2015_18_54.nc
2015_18_6.nc
2015_18_7.nc
2015_19_52.nc
2015_1_44.nc
2015_20_18.nc
2015_20_22.nc
2015_20_37.nc
2015_21_57.nc
2015_21_58.nc
2015_22_26.nc
2015_22_5.nc
2015_22_56.nc
2015_23_16.nc
2015_24_43.nc
2015_24_56.nc
2015_24_59.nc
2015_25_16.nc
2015_26_54.nc
2015_27_19.nc
2015_28_16.nc
2015_28_18.nc
2015_29_8.nc
2015_2_30.nc
2015_30_9.nc
2015_31_55.nc
2015_32_56.nc
2015_33_39.nc
2015_33_55.nc
2015_34_52.nc
2015_35_47.nc
2015_36_45.nc
2015_37_41.nc
2015_37_44.nc
2015_38_48.nc
2015_39_45.nc
2015_3_44.nc
2015_41_48.nc
2015_42_49.nc
2

In [2]:
def get_data_from_netcdf(latitude,longitude,fire_date,window):
    # dictionary for met data
    met_data = {}
    # initialise constants
    latitude0 = -9.75
    longitude0 = 109.5
    lat_resolution = 0.75
    lon_resolution = 0.75
    nc_file_path = "/home/peter/hackfest/"
    # get the year
    year = fire_date.year
    # get the latitude and longitude indices
    lat_index = int(((latitude0-latitude)/lat_resolution)+0.5)
    lon_index = int(((longitude-longitude0)/lon_resolution)+0.5)
    # get the name of the netCDF file containing the data
    nc_file_name = str(year)+"_"+str(lat_index)+"_"+str(lon_index)+".nc"
    nc_full_name = nc_file_path+nc_file_name
    print " File name is: ",nc_full_name
    if not os.path.exists(nc_full_name):
        print " File "+nc_full_name+" not found, skipping ..."
        return met_data
    # get the data
    print " Opening file "+nc_file_name
    nc_file = netCDF4.Dataset(nc_full_name,'r')
    nc_time = nc_file.variables["time"][:]
    nc_time_units = getattr(nc_file.variables["time"],"units")
    dt_utc = netCDF4.num2date(nc_time,nc_time_units)
    start_date = fire_date - datetime.timedelta(days=window["before"])
    end_date = fire_date + datetime.timedelta(days=window["after"])
    if start_date<datetime.datetime(year,1,1,0,0,0):
        print " start_date before beginning of year, skipping ..."
        return met_data
    si = [n for n,dt in enumerate(dt_utc) if dt>start_date ][0]
    ei = [n for n,dt in enumerate(dt_utc) if dt>end_date ][0]
    met_data["DateTime"] = dt_utc[si:ei+1]
    met_data["Fsd"] = nc_file.variables["Fsd"][si:ei+1]
    met_data["Ta"] = nc_file.variables["Ta"][si:ei+1]
    met_data["RH"] = nc_file.variables["RH"][si:ei+1]
    met_data["Ws"] = nc_file.variables["Ws"][si:ei+1]
    met_data["Wd"] = nc_file.variables["Wd"][si:ei+1]
    met_data["Precip"] = nc_file.variables["Precip"][si:ei+1]
    nc_file.close()
    return met_data

In [3]:
# get the filename
file_path = "/home/peter/hackfest/"
year = 2015
csv_filename = file_path+"hotspot_"+str(year)+".csv"
erai_name = file_path+"ERAI_"+str(year)+".nc"
# sniff the file to see if we can find the dialect
csv_file = open(csv_filename,'rb')
dialect = csv.Sniffer().sniff(csv_file.readline(), [' ',',','\t'])
# rewind file
csv_file.seek(0)
csv_reader = csv.reader(csv_file,dialect)
# get the header line
header = csv_reader.next()
# close the file
csv_file.close()
# get a list of columns to be read
csv_list = ["load_dt","latitude","longitude"]
col_list = [header.index(item) for item in csv_list]
# read the csv file using numpy's genfromtxt
skip = 1
# define the missing values and the value with which to fill them
missing_values = {}
filling_values = {}
for item in col_list:
    missing_values[item] = ["NA","N/A","NAN","#NAME?","#VALUE!","#DIV/0!","#REF!"]
    filling_values[item] = float(-9999)
# read the CSV file
fire_data = numpy.genfromtxt(csv_filename,delimiter=dialect.delimiter,skip_header=skip,
                        names=header,usecols=col_list,missing_values=missing_values,
                        filling_values=filling_values,dtype=None)

In [ ]:
plot_data = True
window = {"before":5,"after":0}
nrecs = len(fire_data["latitude"])
# test with first 5
# normal operation would use nrecs
nplots = 0
for i in range(nrecs):
    latitude = fire_data["latitude"][i]
    longitude = fire_data["longitude"][i]
    fire_date = dateutil.parser.parse(fire_data["load_dt"][i])
    met_data = get_data_from_netcdf(latitude,longitude,fire_date,window)
    if "DateTime" not in met_data: continue
    if plot_data and nplots<10:
        nplots = nplots+1
        fig = plt.figure()
        plt.subplot(311)
        plt.plot(met_data["DateTime"],met_data["Fsd"])
        plt.subplot(312)
        plt.plot(met_data["DateTime"],met_data["Ta"])
        plt.subplot(313)
        plt.plot(met_data["DateTime"],met_data["RH"])
        plt.show()

 File name is:  /home/peter/hackfest/2015_7_47.nc
 File /home/peter/hackfest/2015_7_47.nc not found, skipping ...
 File name is:  /home/peter/hackfest/2015_8_36.nc
 Opening file 2015_8_36.nc
 start_date before beginning of year, skipping ...
 File name is:  /home/peter/hackfest/2015_10_48.nc
 Opening file 2015_10_48.nc
 start_date before beginning of year, skipping ...
 File name is:  /home/peter/hackfest/2015_11_18.nc
 Opening file 2015_11_18.nc
 start_date before beginning of year, skipping ...
 File name is:  /home/peter/hackfest/2015_11_18.nc
 Opening file 2015_11_18.nc
 start_date before beginning of year, skipping ...
 File name is:  /home/peter/hackfest/2015_12_17.nc
 Opening file 2015_12_17.nc
 start_date before beginning of year, skipping ...
 File name is:  /home/peter/hackfest/2015_16_9.nc
 Opening file 2015_16_9.nc
 start_date before beginning of year, skipping ...
 File name is:  /home/peter/hackfest/2015_16_9.nc
 Opening file 2015_16_9.nc
 start_date before beginning of y